# Analysing the cleaned OCOD dataset

This is causing problems and needs to be fixed needs the preceeding code
'hillfield mansions and tagore house'	

In [36]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import pandas as pd
import numpy as np
import os
import re
import io
import zipfile
from helper_functions import *



In [2]:
#!pip install thefuzz[speedup]
#from thefuzz import process #an alternative could jellyfish, nltk, diff-match-patch

In [2]:
target_post_area = "Data/ONSPD_NOV_2021_UK.csv"

with zipfile.ZipFile("/tf/empty_homes_data/" + "ONSPD_NOV_2021_UK.zip") as zf:
    with io.TextIOWrapper(zf.open(target_post_area), encoding = 'latin-1') as f:
        postcode_district_lookup = pd.read_csv(f)[['pcds','oslaua','lsoa11', 'msoa11', 'ctry']]
        postcode_district_lookup = postcode_district_lookup[(postcode_district_lookup['ctry'] == 'E92000001') | (postcode_district_lookup['ctry'] == 'W92000004')]
        postcode_district_lookup.rename(columns = {'pcds':'postcode2',
                                                  'oslaua':'lad11cd',
                                                  'lsoa11': 'lsoa11cd',
                                                  'msoa11':'msoa11cd'}, inplace = True)
        #spaces are removed because I don't know if the formatting is the same in the two datasets
        postcode_district_lookup['postcode2']= postcode_district_lookup['postcode2'].str.lower().str.replace(r"\s", r"", regex = True)
        postcode_district_lookup.drop('ctry',  axis =1, inplace = True)


<ipython-input-2-4f134914fcc7>:5: DtypeWarning: Columns (18,31,40,45) have mixed types. Specify dtype option on import or set low_memory=False.
  postcode_district_lookup = pd.read_csv(f)[['pcds','oslaua','lsoa11', 'msoa11', 'ctry']]


In [3]:
ocod_data =  pd.read_csv("/tf/empty_homes_data/OCOD_cleaned_expanded.csv")

##add in the geographic area data like lsoa etc
ocod_data['postcode2'] = ocod_data['postcode'].str.lower().str.replace("\s", "")

ocod_data = ocod_data.merge(postcode_district_lookup, 'left', left_on = "postcode2", right_on = "postcode2")

ocod_data['street_name'] = ocod_data['street_name'].str.replace(r"^ +| +$", r"", regex=True)


#This stripped out versionof street name is used several times throughout the notebook
ocod_data['street_name2'] = ocod_data.loc[:,'street_name'].str.replace(r"'", "", regex = True).\
    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)

<ipython-input-3-f41646608384>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  ocod_data['postcode2'] = ocod_data['postcode'].str.lower().str.replace("\s", "")


In [5]:
ocod_data

,Unnamed: 0,title_number,within_title_id,unique_id,within_larger_title,tenure,unit_id,unit_type,building_name,street_number,...,county,region,multiple_address_indicator,price_paid,property_address,postcode2,lad11cd,lsoa11cd,msoa11cd,street_name2
0,0,CB400630,1,CB400630-1,True,Freehold,NaN,NaN,NaN,2,...,CITY OF PETERBOROUGH,EAST ANGLIA,N,NaN,"2-24 (even) Miller Way, 15-25 Hammonds Drive (...",NaN,NaN,NaN,NaN,millerway
1,1,CB400630,2,CB400630-2,True,Freehold,NaN,NaN,NaN,4,...,CITY OF PETERBOROUGH,EAST ANGLIA,N,NaN,"2-24 (even) Miller Way, 15-25 Hammonds Drive (...",NaN,NaN,NaN,NaN,millerway
2,2,CB400630,3,CB400630-3,True,Freehold,NaN,NaN,NaN,6,...,CITY OF PETERBOROUGH,EAST ANGLIA,N,NaN,"2-24 (even) Miller Way, 15-25 Hammonds Drive (...",NaN,NaN,NaN,NaN,millerway
3,3,CB400630,4,CB400630-4,True,Freehold,NaN,NaN,NaN,8,...,CITY OF PETERBOROUGH,EAST ANGLIA,N,NaN,"2-24 (even) Miller Way, 15-25 Hammonds Drive (...",NaN,NaN,NaN,NaN,millerway
4,4,CB400630,5,CB400630-5,True,Freehold,NaN,NaN,NaN,10,...,CITY OF PETERBOROUGH,EAST ANGLIA,N,NaN,"2-24 (even) Miller Way, 15-25 Hammonds Drive (...",NaN,NaN,NaN,NaN,millerway
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162052,87567,AGL465007,1,AGL465007-1,False,Leasehold,807,flat,heritage tower,118,...,GREATER LONDON,GREATER LONDON,N,479950.0,"Flat 807, Heritage Tower, 118 East Ferry Road,...",e143nw,E09000030,E01004215,E02000891,eastferryroad
162053,87568,AGL465195,1,AGL465195-1,False,Leasehold,NaN,NaN,ormond house,4t,...,GREATER LONDON,GREATER LONDON,N,NaN,"4th Floor, Ormond House, 63 Queen Victoria Str...",ec4n4ua,E09000001,E01032739,E02000001,queenvictoriastreet
162054,87569,AGL473126,1,AGL473126-1,False,Leasehold,NaN,NaN,NaN,155,...,GREATER LONDON,GREATER LONDON,N,NaN,"155 Bishopsgate, London (EC2M 3AD)",ec2m3ad,E09000001,E01032739,E02000001,bishopsgate
162055,87570,AGL475468,1,AGL475468-1,False,Leasehold,NaN,NaN,NaN,NaN,...,GREATER LONDON,GREATER LONDON,N,NaN,"part of Tenth Floor, 6 Bevis Marks, London (EC...",ec3a7ba,E09000001,E01032739,E02000001,NaN


## Using price paid data to match names

The land registry does not use standardised LAD codes or names and 
the LAD names it uses appear to be wrong sometimes. I need to know the LADs so that I only try road matching within local authorities to minimise the chance of having the same road twice. To get around this I will use the substantially larger database of the price paid data to get all the land registry district names and match them to the onsp using the postcodes. This works as there are a large number of sales in each district most of them will have a postcode. There are cases where the wrong district or postcode is applied meaning a single district name can have two or more lad11cd's, to solve this I simply take the lad11cd with the largest number of counts.

The resulting OCOD data frame has a LAD11CD for each entry, and thus allows the fuzzy road matching to work effectively

In [4]:
#https://www.gov.uk/guidance/about-the-price-paid-data#explanations-of-column-headers-in-the-ppd
price_paid_headers = ['Transaction unique identifier', 'Price', 'Date of Transfer', 'Postcode', 'Property Type', 
                     'Old New', 'Duration', 'PAON', 'SAON', 'Street',  'Locality', 'Town', 'District', 'County',
                     'PPD Category Type', 'Record Status - monthly file only']
#clean up to make working with them easier
price_paid_headers = [x.lower().replace(' ', '_') for x in price_paid_headers]

##
## Here we add in the lsoa11cd msoa11cd and lad11cd using postcode
##

#price_paid_df = pd.read_csv('/tf/empty_homes_data/price_paid_files/pp-2021.csv', names = price_paid_headers)
price_paid_df = pd.concat([ pd.read_csv('/tf/empty_homes_data/price_paid_files/'+x, names = price_paid_headers) for x in os.listdir('/tf/empty_homes_data/price_paid_files')])

price_paid_df['street'] = price_paid_df['street'].str.lower()

price_paid_df['street_name2'] = price_paid_df.loc[:,'street'].str.replace(r"'", "", regex = True).\
    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)

price_paid_df['locality'] = price_paid_df['locality'].str.lower()
price_paid_df['paon'] = price_paid_df['paon'].str.lower()

price_paid_df['postcode2'] = price_paid_df['postcode'].str.lower().str.replace(r"\s", r"", regex=True)

price_paid_df = price_paid_df.merge(postcode_district_lookup, 'left', left_on = "postcode2", right_on = "postcode2")

##
## Not all observations have a postcode and some postcodes are erroneous or otherwise cannot be found in the postcode database
## These entries need to be found ny joining districts
##

#when there are multiples take the lad11cd with the largest number of counts
lad_lookup = price_paid_df[['district', 'lad11cd']].dropna().groupby(['district', 'lad11cd']).size().reset_index()
lad_lookup.rename(columns = {0:'counts'}, inplace = True)
lad_lookup = lad_lookup.sort_values('counts', ascending=False).groupby('lad11cd').first().reset_index()
lad_lookup.drop('counts', axis = 1, inplace = True)

temp = ocod_data

#only take data that is missing the lad code. If you include only is missing postcode, bad postcode entries also get excluded leaving a small number of
## blanks in the final dataset
temp = temp[temp['lad11cd'].isna()]

temp = temp.drop('lad11cd', axis = 1)

temp = temp.merge(lad_lookup, left_on = "district", right_on = "district")

temp['lad11cd'].isna().sum() #there are no na values showing all districts now have a lad code

#join the ocod data back together again
ocod_data = pd.concat( [temp, ocod_data[~ocod_data['lad11cd'].isna()]])

#tidy up
#del temp
#del lad_lookup
#del price_paid_df

In [47]:
price_paid_df

,transaction_unique_identifier,price,date_of_transfer,postcode,property_type,old_new,duration,paon,saon,street,...,town,district,county,ppd_category_type,record_status_-_monthly_file_only,street_name2,postcode2,lad11cd,lsoa11cd,msoa11cd
0,{64342BFF-702A-422C-E053-6C04A8C0FB8A},555000,2017-09-18 00:00,PO19 1LB,O,N,L,,NaN,north street,...,CHICHESTER,CHICHESTER,WEST SUSSEX,B,A,northstreet,po191lb,E07000225,E01031490,E02006570
1,{64342BFF-702B-422C-E053-6C04A8C0FB8A},25000,2017-09-29 00:00,NaN,O,Y,L,ifield car park,PLOT 55,charlwood road,...,CRAWLEY,HORSHAM,WEST SUSSEX,B,A,charlwoodroad,NaN,NaN,NaN,NaN
2,{64342BFF-702C-422C-E053-6C04A8C0FB8A},177500,2017-04-07 00:00,PO19 7BJ,O,N,F,glenmore business park portfield works,UNIT 6,chichester by pass,...,CHICHESTER,CHICHESTER,WEST SUSSEX,B,A,chichesterbypass,po197bj,E07000225,E01031513,E02006572
3,{64342BFF-702D-422C-E053-6C04A8C0FB8A},90000,2017-10-09 00:00,BN43 5DB,O,N,L,,NaN,high street,...,SHOREHAM-BY-SEA,ADUR,WEST SUSSEX,B,A,highstreet,bn435db,E07000223,E01031364,E02006538
4,{64342BFF-702E-422C-E053-6C04A8C0FB8A},190000,2017-10-10 00:00,PO21 1QD,O,N,F,,NaN,station road,...,BOGNOR REGIS,ARUN,WEST SUSSEX,B,A,stationroad,po211qd,E07000224,E01031436,E02006558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4812242,{C3C3F9B5-7995-362B-E053-6B04A8C03ACC},220000,2021-03-08 00:00,TR11 2RR,T,N,F,,NaN,pengarth rise,...,FALMOUTH,CORNWALL,CORNWALL,A,A,pengarthrise,tr112rr,E06000052,E01018806,E02003916
4812243,{C3C3F9B5-7996-362B-E053-6B04A8C03ACC},430000,2021-02-05 00:00,PL15 8SS,D,N,F,,NaN,well meadow,...,LAUNCESTON,CORNWALL,CORNWALL,A,A,wellmeadow,pl158ss,E06000052,E01018968,E02003934
4812244,{C3C3F9B5-7997-362B-E053-6B04A8C03ACC},236500,2021-03-12 00:00,TR21 0NE,F,N,L,mount flagon,ANNETT,NaN,...,ISLES OF SCILLY,ISLES OF SCILLY,ISLES OF SCILLY,A,A,NaN,tr210ne,E06000053,E01019077,E02006781
4812245,{C3C3F9B5-7998-362B-E053-6B04A8C03ACC},185050,2021-03-09 00:00,TR27 4DJ,T,N,F,,NaN,commercial road,...,HAYLE,CORNWALL,CORNWALL,A,A,commercialroad,tr274dj,E06000052,E01018982,E02003944


## Load Voa and get business postcodes

This alllows businesses to be identified, and also missing lsoa to be added via the street name

In [5]:
VOA_headers_raw= ["Incrementing Entry Number", "Billing Authority Code", "NDR Community Code", 
 "BA Reference Number", "Primary And Secondary Description Code", "Primary Description Text",
"Unique Address Reference Number UARN", "Full Property Identifier", "Firms Name", "Number Or Name",
"Street", "Town", "Postal District", "County", "Postcode", "Effective Date", "Composite Indicator",
 "Rateable Value", "Appeal Settlement Code", "Assessment Reference", "List Alteration Date", "SCAT Code And Suffix",
 "Sub Street level 3", "Sub Street level 2", "Sub Street level 1", "Case Number", 
 "Current From Date", "Current To Date", 
]

#set to lower and replace spaces with underscore to turn the names into appropriate column names
VOA_headers = [x.lower().replace(" ", "_") for x in VOA_headers_raw]


voa_businesses =  pd.read_csv('/tf/empty_homes_data/' +
                    'uk-englandwales-ndr-2017-listentries-compiled-epoch-0029-baseline-csv.csv',
                   sep = "*",
                   encoding_errors= 'ignore',
                    header=None,
                   names = VOA_headers,
                    index_col = False,
                    #usecols = list(range(1,28))
                   )
voa_businesses['postcode'] = voa_businesses['postcode'].str.lower()
voa_businesses['street'] = voa_businesses['street'].str.lower()

#voa_businesses['street_name2'] = voa_businesses['street'].str.replace(r"'", "", regex = True).\
#    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)
voa_businesses['street_name2'] = voa_businesses.loc[:,'street'].str.replace(r"'", "", regex = True).\
    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)

#this removes advertising hordings which are irrelevant
voa_businesses = voa_businesses.loc[voa_businesses['primary_description_text'].str.contains("ADVERTISING")==False,:]
#remove several kinds of car parking space
voa_businesses = voa_businesses[~voa_businesses['primary_and_secondary_description_code'].isin(['C0', 'CP','CP1', 'CX', 'MX'])]
##
##
## Warning this removes a large amount of columns, these may be interesting for some people
##
##
voa_businesses = voa_businesses.iloc[:,4:15]
#Extract the street number
#replace unit numbers with nothing to avoid accidentally using them as street numbers
voa_businesses['street_number'] = voa_businesses['number_or_name'].str.replace(r"(unit|suite)\s\d+(,)?", "", regex = True).str.extract(r"(\b[0-9\-]+$)")
voa_businesses['postcode2'] = voa_businesses['postcode'].str.lower().str.replace("\s", "", regex = True)

#add in postcode data and LSOA etc data, this is useful for a range of tasks
voa_businesses = voa_businesses.merge(postcode_district_lookup, left_on = 'postcode2', right_on = "postcode2")

#Create a dataframe that contains the counts of businesses per postcode
postcode_counts_voa = voa_businesses.groupby('postcode').size().reset_index(name = 'business_counts')

ocod_data = pd.merge(ocod_data, postcode_counts_voa, on = "postcode", how = "left")
ocod_data["business_counts"] = ocod_data["business_counts"].fillna(0)

voa_businesses['street_name2'] = voa_businesses.loc[:,'street'].str.replace(r"'", "", regex = True).\
    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)
#del voa_businesses

#this is quite large and no longer necessary so is removed from the workspace to save memory
#del postcode_district_lookup

<ipython-input-5-8476be09a14b>:14: DtypeWarning: Columns (1,2,6,12,13,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  voa_businesses =  pd.read_csv('/tf/empty_homes_data/' +


In [33]:
voa_businesses

,primary_and_secondary_description_code,primary_description_text,unique_address_reference_number_uarn,full_property_identifier,firms_name,number_or_name,street,town,postal_district,county,postcode,street_number,postcode2,lad11cd,lsoa11cd,msoa11cd,street_name2
0,MH,SURGERY AND PREMISES,4159370000,"MEDICAL CENTRE, ABBEYFIELDS, ELSTOW, BEDFORD",NaN,MEDICAL CENTRE,abbeyfields,BEDFORD,ELSTOW,NaN,mk42 9gp,NaN,mk429gp,E06000055,E01017518,E02003629,abbeyfields
1,EL,SCHOOL AND PREMISES,8871511000,"ELSTOW SCHOOL, ABBEYFIELDS, ELSTOW, BEDFORD",NaN,ELSTOW SCHOOL,abbeyfields,BEDFORD,ELSTOW,NaN,mk42 9gp,NaN,mk429gp,E06000055,E01017518,E02003629,abbeyfields
2,IF,FACTORY AND PREMISES,287299011,"5, ADAMS CLOSE, KEMPSTON, BEDFORD",NaN,5,adams close,BEDFORD,KEMPSTON,NaN,mk42 7je,5,mk427je,E06000055,E01017516,E02003633,adamclose
3,IF3,WORKSHOP AND PREMISES,456757000,"WOBURN HOUSE 3, ADAMS CLOSE, KEMPSTON, BEDFORD",NaN,WOBURN HOUSE 3,adams close,BEDFORD,KEMPSTON,NaN,mk42 7je,3,mk427je,E06000055,E01017516,E02003633,adamclose
4,IF,FACTORY AND PREMISES,287049011,"1, ADAMS CLOSE, KEMPSTON, BEDFORD",NaN,1,adams close,BEDFORD,KEMPSTON,NaN,mk42 7je,1,mk427je,E06000055,E01017516,E02003633,adamclose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2319291,CO,LABORATORY AND PREMISES,13038764000,"LAB 1, THE EPICENTRE, ENTERPRISE WAY, WITHERSF...",NaN,LAB 1,enterprise way,HAVERHILL,WITHERSFIELD,SUFFOLK,cb9 7lr,1,cb97lr,E07000245,E01030151,E02006283,enterpriseway
2319292,CO,OFFICES AND PREMISES,13171840000,"OFFICE 1-2, THE EPICENTRE, ENTERPRISE WAY, WIT...",NaN,OFFICE 1-2,enterprise way,HAVERHILL,WITHERSFIELD,SUFFOLK,cb9 7lr,1-2,cb97lr,E07000245,E01030151,E02006283,enterpriseway
2319293,CO,OFFICE AND PREMISES,13153535000,"OFFICE 6, THE EPICENTRE, ENTERPRISE WAY, WITHE...",NaN,OFFICE 6,enterprise way,HAVERHILL,WITHERSFIELD,SUFFOLK,cb9 7lr,6,cb97lr,E07000245,E01030151,E02006283,enterpriseway
2319294,CO,OFFICE AND PREMISES,13151982000,"MEETING ROOM G18, THE EPICENTRE, ENTERPRISE WA...",NaN,MEETING ROOM G18,enterprise way,HAVERHILL,WITHERSFIELD,SUFFOLK,cb9 7lr,NaN,cb97lr,E07000245,E01030151,E02006283,enterpriseway


In [8]:
# street = 'miller way'
# lad11cd ='E06000031'

# def lsoa_match_street(price_paid_df, street, lad11cd):
#     #identifies the lsoa of an address suing the street and district only.
#     #returns the lsoa11cd
#     #If a street name is associated with multiple addresses the function returns NaN
#     #street: string. the name of the targe street
#     #lad11cd: the lad code for the target lad

#     #subset by road and district
#     temp = price_paid_df[(price_paid_df['street']==street) & (price_paid_df['lad11cd']== lad11cd)]

#     lsoa_codes = temp['lsoa11cd'].unique()
#     #locality_names = temp['locality'].unique()

#     if len(lsoa_codes) ==1:
#         out = lsoa_codes[0]
#     else:
#         out = np.nan
        
#     return(out)

# lsoa_match_street(price_paid_df, street, lad11cd)

In [9]:
# voa_businesses['street_name2'] = voa_businesses.loc[:,'street'].str.replace(r"'", "", regex = True).\
#     str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)

voa_businesses.columns


Index(['primary_and_secondary_description_code', 'primary_description_text',
       'unique_address_reference_number_uarn', 'full_property_identifier',
       'firms_name', 'number_or_name', 'street', 'town', 'postal_district',
       'county', 'postcode', 'street_number', 'postcode2', 'lad11cd',
       'lsoa11cd', 'msoa11cd', 'street_name2'],
      dtype='object')

In [6]:
##
##This process is quite convoluted and there is certainly a more efficient and pythonic way
## however the order within each filling method is important to ensure that there are no duplicates
## as this causes the OCOD dataset to grow with duplicates
##

#replce the missing lsoa using street matching

temp_lsoa = pd.concat([
    price_paid_df[['street_name2', 'lad11cd', 'lsoa11cd']], voa_businesses[['street_name2', 'lad11cd', 'lsoa11cd']]
                      ]).dropna(axis = 0, how = 'any', inplace = False)

temp = temp_lsoa.groupby(['street_name2', 'lad11cd', 'lsoa11cd']).size().reset_index().groupby(['street_name2', 'lad11cd']).size()\
.reset_index().rename(columns = {0:'counts'})

temp = temp[temp['counts']==1].merge(temp_lsoa.drop_duplicates(), 
                                     how = "left", on = ['street_name2', 'lad11cd']).rename(columns ={'lsoa11cd':'lsoa_street'})

ocod_data = ocod_data.merge(temp[['lsoa_street', "street_name2", "lad11cd"]], how = "left", on = ['street_name2', 'lad11cd'])


#replace the missing lsoa using building matching

temp = price_paid_df

temp['paon'] = temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)

temp = temp.groupby(['paon', 'lad11cd', 'lsoa11cd']).size().reset_index()

temp = temp[temp['paon'].str.len()!=0].groupby(['paon', 'lad11cd']).size().reset_index().rename(columns = {0:'counts'})

pp_temp = price_paid_df[['paon', 'lad11cd', 'lsoa11cd']]

pp_temp['paon'] = pp_temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)

pp_temp = pp_temp.drop_duplicates()

temp = temp[temp['counts']==1].merge(pp_temp, 
                                     how = "left", on = ['paon', 'lad11cd'])

temp = temp.rename(columns ={'lsoa11cd':'lsoa_building'})

ocod_data = ocod_data.merge(temp[['lsoa_building', "paon", "lad11cd"]], 
                       how = "left", 
                       left_on =  ['building_name', 'lad11cd'], 
                       right_on = ['paon', 'lad11cd'])


<ipython-input-6-55f6e4720c85>:26: FutureWarning: The default value of regex will change from True to False in a future version.
  temp['paon'] = temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)
<ipython-input-6-55f6e4720c85>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  pp_temp['paon'] = pp_temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)
<ipython-input-6-55f6e4720c85>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_temp['paon'] = pp_temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)


In [7]:
#I appreciate this is an almost artistically silly way of doing this. 
for x in ['lsoa_street', 'lsoa_building']:#, 'lsoa_business']:
    ocod_data.loc[ocod_data['lsoa11cd'].isnull(), 'lsoa11cd'] = ocod_data[x][ocod_data['lsoa11cd'].isnull()] 

In [346]:
ocod_data['lsoa11cd'].isnull().sum()/ocod_data.shape[0]

0.09882594676222482

In [8]:
#after all other lsoa adding methods are completed
#all nested properties with missing lsoa have the lsoa of the other properties within their group added

temp = ocod_data.loc[(ocod_data['lsoa11cd'].notnull()) & (ocod_data['within_larger_title']==True) ,['lsoa11cd', 'title_number']].\
groupby(['lsoa11cd', 'title_number']).size().reset_index()
temp = temp[['lsoa11cd', 'title_number']].rename(columns = {'lsoa11cd':'lsoa_nested'})

#there are a small number of nested addresses where there are multiple lsoa this prevents increasing the number of observations with these duplicates
#I don't think it matters if a ver observations are in neighbouring lsoa, the general spatial coherence is maintained
temp = temp.groupby('title_number')['lsoa_nested'].first().reset_index()


ocod_data = ocod_data.merge(temp[['title_number', 'lsoa_nested']], 
                       how = "left",
                           on = "title_number")

ocod_data.loc[ocod_data['lsoa11cd'].isnull(), 'lsoa11cd'] = ocod_data['lsoa_nested'][ocod_data['lsoa11cd'].isnull()] 


In [328]:
filled_lsoa_list = []

unique_lad_codes = ocod_data[ocod_data['street_name'].notnull() & ocod_data['street_number'].notnull() & ocod_data['lsoa11cd'].isnull()]['lad11cd'].unique()

for target_lad in unique_lad_codes:
    print(target_lad)
    #subset to the relevat rows within a single lad
    missing_lsoa_df = ocod_data[ocod_data['street_name'].notnull() & ocod_data['street_number'].notnull() & ocod_data['lsoa11cd'].isnull() & (ocod_data['lad11cd']==target_lad)]
    missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)

    target_street_names = missing_lsoa_df['street_name2'].unique()

    temp_lsoa = pd.concat([
        #the price paid data with names changed
        price_paid_df[price_paid_df['street_name2'].isin(target_street_names )  & 
                                    (price_paid_df['lad11cd']==target_lad) ].rename(columns = {'paon':'street_number'}), 
      #voa data added in                
                      voa_businesses[(voa_businesses['lad11cd']==target_lad)]]
                                                    )[['street_name2', 'street_number', 'lsoa11cd', 'lad11cd']].dropna(axis = 0, how = 'any', inplace = False)

    temp_lsoa.loc[:,'street_number2'] = temp_lsoa.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
    
    temp_lsoa  = create_all_street_addresses(temp_lsoa[temp_lsoa['street_name2'].isin(target_street_names ) & 
                                            temp_lsoa['street_number2'].notnull()], 
                                       target_lad, 
                                       ['street_name2', 'street_number2', 'lsoa11cd'])
    

    for target_road in target_street_names:
        print(target_road)
        missing_lsoa_road = missing_lsoa_df[missing_lsoa_df['street_name2']== target_road ]
        temp_road = temp_lsoa[temp_lsoa['street_name2'] ==target_road ]

        if len(temp_road)>0:
            missing_lsoa_road['lsoa11cd'] = [street_number_to_lsoa(temp_road, int(missing_lsoa_road.iloc[missing_lsoa_row]['street_number2'])) 
                                          for missinf_lsoa_row 
                                          in range(0, len(missing_lsoa_road))]
            filled_lsoa_list = filled_lsoa_list + [missing_lsoa_road]

E06000031


<ipython-input-328-489973ab39f8>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
<ipython-input-328-489973ab39f8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
<ipython-input-328-489973ab39f8>:25: FutureWarning: The default value of regex will change from True to False in a future version.
  temp_lsoa.loc[:,'street_number2'] = temp_lsoa.loc[:,'street_number'].str.replace(r"^.*(?=\b

i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
broadway
E07000032


<ipython-input-328-489973ab39f8>:25: FutureWarning: The default value of regex will change from True to False in a future version.
  temp_lsoa.loc[:,'street_number2'] = temp_lsoa.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
<ipython-input-328-489973ab39f8>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
<ipython-input-328-489973ab39f8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b

albionclose
colliergrove
gadfieldgrove
E08000019


...

<ipython-input-328-489973ab39f8>:25: FutureWarning: The default value of regex will change from True to False in a future version.
  temp_lsoa.loc[:,'street_number2'] = temp_lsoa.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
<ipython-input-328-489973ab39f8>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
<ipython-input-328-489973ab39f8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b

mountsideplace
E07000215


<ipython-input-328-489973ab39f8>:25: FutureWarning: The default value of regex will change from True to False in a future version.
  temp_lsoa.loc[:,'street_number2'] = temp_lsoa.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
<ipython-input-328-489973ab39f8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_road['lsoa11cd'] = [street_number_to_lsoa(temp_road, int(missing_lsoa_road.iloc[missing_lsoa_row]['street_number2']))
<ipython-input-328-489973ab39f8>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(

byerlane
thesquare
E07000166
marketplace


<ipython-input-328-489973ab39f8>:25: FutureWarning: The default value of regex will change from True to False in a future version.
  temp_lsoa.loc[:,'street_number2'] = temp_lsoa.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)


In [332]:
temp_lsoa = pd.concat(filled_lsoa_list)

In [344]:
ocod_data = pd.concat([ocod_data[~ocod_data['unique_id'].isin(temp_lsoa['unique_id'])], temp_lsoa ] )
ocod_data

,Unnamed: 0,title_number,within_title_id,unique_id,within_larger_title,tenure,unit_id,unit_type,building_name,street_number,...,lsoa11cd,msoa11cd,street_name2,lad11cd,business_counts,lsoa_street,lsoa_building,paon,lsoa_nested,street_number2
0,0,CB400630,1,CB400630-1,True,Freehold,NaN,NaN,NaN,2,...,E01015611,NaN,millerway,E06000031,0.0,E01015611,NaN,NaN,E01015611,NaN
1,1,CB400630,2,CB400630-2,True,Freehold,NaN,NaN,NaN,4,...,E01015611,NaN,millerway,E06000031,0.0,E01015611,NaN,NaN,E01015611,NaN
2,2,CB400630,3,CB400630-3,True,Freehold,NaN,NaN,NaN,6,...,E01015611,NaN,millerway,E06000031,0.0,E01015611,NaN,NaN,E01015611,NaN
3,3,CB400630,4,CB400630-4,True,Freehold,NaN,NaN,NaN,8,...,E01015611,NaN,millerway,E06000031,0.0,E01015611,NaN,NaN,E01015611,NaN
4,4,CB400630,5,CB400630-5,True,Freehold,NaN,NaN,NaN,10,...,E01015611,NaN,millerway,E06000031,0.0,E01015611,NaN,NaN,E01015611,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57391,27419,NT214819,1,NT214819-1,False,Freehold,NaN,NaN,NaN,27 to 33,...,E01028085,NaN,highroad,E07000172,0.0,NaN,NaN,NaN,NaN,33
57438,65367,ON137736,1,ON137736-1,False,Freehold,NaN,NaN,NaN,2-8,...,E01028763,NaN,highstreet,E07000181,0.0,NaN,NaN,NaN,NaN,8
57458,10412,NT73623,1,NT73623-1,False,Freehold,NaN,NaN,NaN,22,...,E01028268,NaN,churchstreet,E07000174,0.0,NaN,NaN,NaN,NaN,22
57634,20195,SY64666,1,SY64666-1,False,Freehold,NaN,NaN,NaN,8 to 14,...,E01030829,NaN,thesquare,E07000215,0.0,NaN,NaN,NaN,NaN,14


In [345]:
#Doing this grouped nested business a second time pushes lsoa ID over 90% which seems good enough for me

#after all other lsoa adding methods are completed
#all nested properties with missing lsoa have the lsoa of the other properties within their group added

temp = ocod_data.loc[(ocod_data['lsoa11cd'].notnull()) & (ocod_data['within_larger_title']==True) ,['lsoa11cd', 'title_number']].\
groupby(['lsoa11cd', 'title_number']).size().reset_index()
temp = temp[['lsoa11cd', 'title_number']].rename(columns = {'lsoa11cd':'lsoa_nested2'})

#there are a small number of nested addresses where there are multiple lsoa this prevents increasing the number of observations with these duplicates
#I don't think it matters if a ver observations are in neighbouring lsoa, the general spatial coherence is maintained
temp = temp.groupby('title_number')['lsoa_nested2'].first().reset_index()


ocod_data = ocod_data.merge(temp[['title_number', 'lsoa_nested2']], 
                       how = "left",
                           on = "title_number")

ocod_data.loc[ocod_data['lsoa11cd'].isnull(), 'lsoa11cd'] = ocod_data['lsoa_nested2'][ocod_data['lsoa11cd'].isnull()] 

In [349]:
test = ocod_data

In [361]:
ocod_data[ocod_data['title_number']=='NGL737065'][['title_number', 'unique_id','property_address','building_name', 'street_name','street_number' , 'lsoa11cd']]

,title_number,unique_id,property_address,building_name,street_name,street_number,lsoa11cd
4254,NGL737065,NGL737065-1,"191 to 211 (odd) Haverstock Hill, 1 to 21 Hill...",NaN,haverstock hill,NaN,NaN
4255,NGL737065,NGL737065-2,"191 to 211 (odd) Haverstock Hill, 1 to 21 Hill...",NaN,haverstock hill,NaN,NaN
4256,NGL737065,NGL737065-3,"191 to 211 (odd) Haverstock Hill, 1 to 21 Hill...",NaN,haverstock hill,NaN,NaN
4257,NGL737065,NGL737065-4,"191 to 211 (odd) Haverstock Hill, 1 to 21 Hill...",NaN,haverstock hill,NaN,NaN
4258,NGL737065,NGL737065-5,"191 to 211 (odd) Haverstock Hill, 1 to 21 Hill...",NaN,haverstock hill,NaN,NaN
4259,NGL737065,NGL737065-6,"191 to 211 (odd) Haverstock Hill, 1 to 21 Hill...",NaN,haverstock hill,NaN,NaN
4260,NGL737065,NGL737065-7,"191 to 211 (odd) Haverstock Hill, 1 to 21 Hill...",NaN,haverstock hill,NaN,NaN
4261,NGL737065,NGL737065-8,"191 to 211 (odd) Haverstock Hill, 1 to 21 Hill...",NaN,haverstock hill,NaN,NaN
4262,NGL737065,NGL737065-9,"191 to 211 (odd) Haverstock Hill, 1 to 21 Hill...",NaN,haverstock hill,NaN,NaN
4263,NGL737065,NGL737065-10,"191 to 211 (odd) Haverstock Hill, 1 to 21 Hill...",NaN,haverstock hill,NaN,NaN


In [20]:
#this is definately the problem then
pd.crosstab(test['lsoa_street'].notnull(),  test['lsoa_building'].notnull())

lsoa_building,False,True
lsoa_street,,
False,81497,15143
True,54452,10485


In [354]:
test2 = test[test['lsoa11cd'].isnull()]
pd.crosstab(test2['property_address'].str.startswith('land') , test2['lsoa_street'].notnull())

lsoa_street,False
property_address,
False,11844
True,4124


In [357]:
pd.crosstab(test['property_address'].str.contains(r"(cinema)|(hotel)|(office)|centre|(\bpub)|(business)|(cafe)|(^shop)|( shop)|(restaurant)|(home)|(^store(s)?\b)|(^storage\\b)|(company)|(ltd)|(limited)|(plc)|(retail)|(leisure)|(industrial)|(hall of)", case = False), test['lsoa11cd'].notnull())

<ipython-input-357-b9539d05818b>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  pd.crosstab(test['property_address'].str.contains(r"(cinema)|(hotel)|(office)|centre|(\bpub)|(business)|(cafe)|(^shop)|( shop)|(restaurant)|(home)|(^store(s)?\b)|(^storage\\b)|(company)|(ltd)|(limited)|(plc)|(retail)|(leisure)|(industrial)|(hall of)", case = False), test['lsoa11cd'].notnull())


lsoa11cd,False,True
property_address,,
False,14843,140277
True,1125,5332


In [350]:
#this is definately the problem then
pd.crosstab(test['street_name'].notnull(),  test['lsoa11cd'].notnull())

lsoa11cd,False,True
street_name,,
False,4896,16553
True,11072,129056


In [351]:
#this is definately the problem then
pd.crosstab(test['street_name'].notnull(),  test['lsoa11cd'].notnull())

lsoa11cd,False,True
street_name,,
False,4896,16553
True,11072,129056


In [353]:
test[test['lsoa11cd'].isnull() & test['street_name'].notnull()].to_csv('/tf/empty_homes_data/delete_me.csv')

In [ ]:
price_paid_df.groupby(['street', 'district', 'locality', 'lsoa11']).size().reset_index().groupby(['street', 'district', 'locality']).\
size().reset_index().rename(columns = {0:'counts'}).sort_values('counts', ascending = False).to_csv('/tf/empty_homes_data/delete_me.csv')

In [ ]:
#95.5% of sets have only a single lsoa, when grouped by street, town, district and locality
#when grouped by only street and district, this number is still 90%
#excluding town the number is still 0.95% but dropping locality gives a match on 91%, therefore using locality is the key
temp = price_paid_df.groupby(['street', 'district', 'locality', 'lsoa11cd']).size().reset_index().groupby(['street', 'district', 'locality']).size()\
.reset_index().rename(columns = {0:'counts'})

temp.groupby('counts').size()/temp.shape[0]


In [31]:
ocod_data.groupby('unit_type').size()

unit_type
airspace              1561
apartment             5188
business               285
cafe                     8
car park space          51
car parking space      448
cinema                   8
flat                 14309
garage                 408
hotel                   78
land                 12487
office                  91
parking space         1107
penthouse               92
plot                  2430
pub                      1
restaurant              18
room                   959
storage                144
store                  175
suite                  134
unit                  3042
dtype: int64

In [33]:
ocod_data.lsoa11cd.isnull().sum()/ocod_data.shape[0] #How can location be given when missing postcode?

0.18041173417690234

In [ ]:
#only 10k have neither postcode nor street name
#This means I can try to geomatch using street name. In the best case only 4% of addresses will not be matched
pd.crosstab(ocod_data.postcode.notnull(), ocod_data.street_name.notnull())#/ocod_data.shape[0]



# VOA matching businesses

In [59]:
voa_businesses[(voa_businesses['street'].str.contains('kensington church street')==True) &  (voa_businesses['lad11cd']=='E09000020')]

,primary_and_secondary_description_code,primary_description_text,unique_address_reference_number_uarn,full_property_identifier,firms_name,number_or_name,street,town,postal_district,county,postcode,street_number,postcode2,lad11cd,lsoa11cd,msoa11cd
1999622,MT1,COMMUNICATION STATION AND PREMISES,7181351000,"VODAFONE SITE (55959) AT 25, KENSINGTON CHURCH...",NaN,VODAFONE SITE (55959) AT 25,kensington church street,LONDON,NaN,NaN,w8 4ll,25,w84ll,E09000020,E01002826,E02000584
1999623,MH,SURGERY AND PREMISES,9489996000,"GRD & 1ST FLR 27A, KENSINGTON CHURCH STREET, L...",NaN,GRD & 1ST FLR 27A,kensington church street,LONDON,NaN,NaN,w8 4ll,NaN,w84ll,E09000020,E01002826,E02000584
1999624,CO,OFFICES AND PREMISES,11740738000,"OFFICE B 1ST FLR 31, KENSINGTON CHURCH STREET,...",NaN,OFFICE B 1ST FLR 31,kensington church street,LONDON,NaN,NaN,w8 4ll,31,w84ll,E09000020,E01002826,E02000584
1999625,CO,OFFICES AND PREMISES,11740737000,"OFFICE A 1ST FLR 31, KENSINGTON CHURCH STREET,...",NaN,OFFICE A 1ST FLR 31,kensington church street,LONDON,NaN,NaN,w8 4ll,31,w84ll,E09000020,E01002826,E02000584
1999626,CS,SHOP AND PREMISES,13312613000,"33, KENSINGTON CHURCH STREET, LONDON",NaN,33,kensington church street,LONDON,NaN,NaN,w8 4ll,33,w84ll,E09000020,E01002826,E02000584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001500,CS,SHOP AND PREMISES,37612082,"36B, KENSINGTON CHURCH STREET, LONDON",NaN,36B,kensington church street,LONDON,NaN,NaN,w8 4bx,NaN,w84bx,E09000020,E01002826,E02000584
2001501,CS,SHOP AND PREMISES,37586082,"104, KENSINGTON CHURCH STREET, LONDON",NaN,104,kensington church street,LONDON,NaN,NaN,w8 4bu,104,w84bu,E09000020,E01002824,E02000584
2001502,CS,SHOP AND PREMISES,37584082,"104A, KENSINGTON CHURCH STREET, LONDON",NaN,104A,kensington church street,LONDON,NaN,NaN,w8 4bu,NaN,w84bu,E09000020,E01002824,E02000584
2001503,CS3,HAIRDRESSING SALON AND PREMISES,37585082,"104C, KENSINGTON CHURCH STREET, LONDON",NaN,104C,kensington church street,LONDON,NaN,NaN,w8 4bu,NaN,w84bu,E09000020,E01002824,E02000584


## Creating a voa expander for individual LADs

In [ ]:
target_lad = 'E08000014'

temp = voa_businesses[voa_businesses['lad11cd'] == target_lad].copy(deep = True)
    
    #remove anything in brackets
temp['street_number'] = temp['street_number'].str.replace(r"\(.+\)", "", regex = True, case = False)
    
    #units often slip in as street numbers, this kills them off
temp.loc[temp['street_number'].str.contains(r"unit|suite", regex = True, case = False)==True, 'street_number'] = np.nan
    
    #replace @ and & with words
temp['street_number'] = temp['street_number'].str.replace(r"@", " at ", regex = True, case = False).str.replace(r"&", " and ", regex = True, case = False)
    
    #replace "-" with spaces with a simple "-"
temp['street_number'] = temp['street_number'].str.replace(r"(\s)?-(\s)?", "-", regex = True, case = False)
    
    #take only things after the last space includes cases where there is no space. Then remove all letters
temp['street_number'] = temp['street_number'].str.extract(r"([^\s]+$)")[0].str.replace(r"([a-z]+)", "", regex = True, case = False)
    #remove dangling hyphens and slashes
temp['street_number'] = temp['street_number'].str.replace(r"(-$)|(^-)|\\|\/", "", regex = True, case = False)
    #replace double hyphen... yes it happens
temp['street_number'] = temp['street_number'].str.replace(r"--", r"-", regex = True, case = False)
temp.loc[temp['street_number'].str.len() == 0, 'street_number'] = np.nan
temp.loc[temp['street_number'].str.contains(r"\.", regex = True)==True, 'street_number'] = np.nan

temp['is_multi'] = temp['street_number'].str.contains(r"-", regex = True)

temp_multi = temp.loc[temp['is_multi']==True]#, ['street_number', 'is_multi']]

#temp_multi['number_filter'] = [find_filter_type(x) for x in temp_multi['street_number']]

i = 0
temp_list = []
for x in temp_multi['street_number']:#.unique():
        print(str(i)+ " "+x)
        i = i+1
        values = [int(x) for x in [x for x in x.split("-")]]
        temp_list = temp_list +[values]

In [ ]:
temp_multi.iloc[248]

In [ ]:
test = ocod_data[(ocod_data['lad11cd']==target_lad)].copy(deep = True)
test['matches_business_address'] = test['street_name'].isin(all_street_addresses['street']) & test['street_number'].isin(all_street_addresses['street_number'])
test['matches_business_address'].sum()

In [ ]:
#filters to a single LAD
    #removes advertising hoardings which are irrelevant
    LAD_biz = voa_businesses.loc[(voa_data['lad11cd']==target_lad) & ~voa_data['primary_description_text'].str.contains("ADVERTISING")].copy(deep = True)
    
    LAD_biz.loc[:,'street_name2'] = LAD_biz['street'].copy(deep=True)
    #remove apostraphe's
    LAD_biz.loc[:,'street_name2'] = LAD_biz.loc[:,'street_name2'].str.replace(r"'", "", regex = True).\
    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)
    
    #subset to target LAD
    ocod_data_road = ocod_data[ocod_data['lad11cd']==target_lad].copy(deep = True)
    #replace nan values to prevent crash    
    
    #create second column
    ocod_data_road['street_name2'] = ocod_data_road['street_name'].copy(deep=True)
    
    #replace nan values to prevent crash    
    ocod_data_road.loc[ocod_data_road.street_name.isna(),'street_name'2] ="xxxstreet name missingxxx"
    #clean street names of common matching errors
    #remove apostraphe's
    #remove trailing 's'
    #remove all spaces
    ocod_data_road['street_name2'] = ocod_data_road['street_name2'].str.replace(r"'", "", regex = True).\
    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)
    
    ocod_data_road['match'] = ocod_data_road['street_name2'].isin(LAD_biz.street_name2.unique())

In [ ]:
street_name = ocod_data.loc[ocod_data['lad11cd'] == target_lad ,'street_name']

In [ ]:
target_lad = 'E08000014'
test = massaged_address_match(ocod_data, voa_businesses, target_lad)

In [59]:
test

,Unnamed: 0,title_number,within_title_id,unique_id,within_larger_title,tenure,unit_id,unit_type,building_name,street_number,...,lad11cd,business_counts,lsoa_business,lsoa_street,lsoa_building,paon,lsoa_nested,street_match,address_match,business_address
0,2,CB247793,3,CB247793-3,True,Freehold,NaN,NaN,NaN,40,...,E06000031,0.0,E01015599,E01015599,NaN,NaN,E01015599,True,True,"40, BRIDGE STREET, PETERBOROUGH"
1,0,CB261784,1,CB261784-1,True,Leasehold,NaN,NaN,NaN,4,...,E06000031,0.0,E01015611,NaN,NaN,NaN,E01015611,True,True,"4, HARVESTER WAY, PETERBOROUGH"
2,38750,CB258582,1,CB258582-1,False,Leasehold,NaN,NaN,NaN,4,...,E06000031,0.0,E01015611,NaN,NaN,NaN,NaN,True,True,"4, HARVESTER WAY, PETERBOROUGH"
3,50983,CB39813,1,CB39813-1,False,Leasehold,NaN,NaN,NaN,4,...,E06000031,0.0,E01015611,NaN,NaN,NaN,NaN,True,True,"4, HARVESTER WAY, PETERBOROUGH"
4,2,CB261784,3,CB261784-3,True,Leasehold,NaN,NaN,NaN,6,...,E06000031,0.0,E01015611,NaN,NaN,NaN,E01015611,True,True,"5/6, HARVESTER WAY, PETERBOROUGH"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,25568,K923145,1,K923145-1,False,Leasehold,NaN,NaN,homepine house,125,...,E07000112,0.0,NaN,NaN,E01024509,homepine house,NaN,True,True,"125, SANDGATE ROAD, FOLKESTONE, KENT"
9,52956,K599843,1,K599843-1,False,Leasehold,NaN,NaN,homepine house,125,...,E07000112,0.0,NaN,NaN,E01024509,homepine house,NaN,True,True,"125, SANDGATE ROAD, FOLKESTONE, KENT"
10,49086,K571690,1,K571690-1,False,Freehold,NaN,NaN,NaN,11,...,E07000112,0.0,E01024537,E01024537,NaN,NaN,NaN,True,True,"11, DUNES ROAD, GREATSTONE, NEW ROMNEY, KENT"
11,52348,K692615,1,K692615-1,False,Freehold,NaN,NaN,NaN,84,...,E07000112,0.0,NaN,NaN,NaN,NaN,NaN,True,True,"GND FLR & BST 84, SANDGATE ROAD, FOLKESTONE, KENT"


In [57]:
pd.crosstab(ocod_data['street_name'][ocod_data['postcode'].isnull()].notnull(), ocod_data['street_number'][ocod_data['postcode'].isnull()].notnull())

street_number,False,True
street_name,,
False,5038,2406
True,22092,38650


In [58]:
pd.crosstab(ocod_data['street_name'][ocod_data['postcode'].isnull()].notnull(), ocod_data['building_name'][ocod_data['postcode'].isnull()].notnull())

building_name,False,True
street_name,,
False,5045,2399
True,50575,10167


In [ ]:
pd.crosstab(ocod_data['street_name'][ocod_data['postcode'].isnull() & ocod_data['street_number'].isnull()].notnull(), 
            ocod_data['building_name'][ocod_data['postcode'].isnull() & ocod_data['street_number'].isnull()].notnull())

In [53]:
all_lads = ocod_data.lad11cd.unique()
#all_lads = all_lads[all_lads != 'E06000031']
#all_lads = all_lads[all_lads != 'E08000017']

matched_lads_list = []

all_lads = [x for x in all_lads if str(x) != 'nan']
#see which roads match a road in voa data set for each local authority
for target_lad in all_lads:
    print(target_lad)
    #temp['matches_business_address'] = business_address_matcher(temp['street_name'], temp['street_number'], voa_businesses, target_lad)
    matched_lads_list = matched_lads_list + [massaged_address_match(ocod_data, voa_businesses, target_lad)]

    #matched_lads_list = [massaged_address_match(ocod_data, voa_businesses, target_lad) for target_lad in all_lads]


E06000031
i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001


/tf/empty_homes_london/helper_functions.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


E07000032
i= 0  expand time,0.001 filter time0.0 make_dataframe_time 0.001


/tf/empty_homes_london/helper_functions.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


E08000019


...

i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
E06000062


/tf/empty_homes_london/helper_functions.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
E07000112


/tf/empty_homes_london/helper_functions.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
E06000053


In [55]:
test  = pd.concat(matched_lads_list)
#test[test['address_match']].to_csv('/tf/empty_homes_data/delete_me.csv')

In [64]:
test#['address_match'].sum()

,Unnamed: 0,title_number,within_title_id,unique_id,within_larger_title,tenure,unit_id,unit_type,building_name,street_number,...,lad11cd,business_counts,lsoa_business,lsoa_street,lsoa_building,paon,lsoa_nested,street_match,address_match,business_address
0,2,CB247793,3,CB247793-3,True,Freehold,NaN,NaN,NaN,40,...,E06000031,0.0,E01015599,E01015599,NaN,NaN,E01015599,True,True,"40, BRIDGE STREET, PETERBOROUGH"
1,0,CB261784,1,CB261784-1,True,Leasehold,NaN,NaN,NaN,4,...,E06000031,0.0,E01015611,NaN,NaN,NaN,E01015611,True,True,"4, HARVESTER WAY, PETERBOROUGH"
2,38750,CB258582,1,CB258582-1,False,Leasehold,NaN,NaN,NaN,4,...,E06000031,0.0,E01015611,NaN,NaN,NaN,NaN,True,True,"4, HARVESTER WAY, PETERBOROUGH"
3,50983,CB39813,1,CB39813-1,False,Leasehold,NaN,NaN,NaN,4,...,E06000031,0.0,E01015611,NaN,NaN,NaN,NaN,True,True,"4, HARVESTER WAY, PETERBOROUGH"
4,2,CB261784,3,CB261784-3,True,Leasehold,NaN,NaN,NaN,6,...,E06000031,0.0,E01015611,NaN,NaN,NaN,E01015611,True,True,"5/6, HARVESTER WAY, PETERBOROUGH"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,25568,K923145,1,K923145-1,False,Leasehold,NaN,NaN,homepine house,125,...,E07000112,0.0,NaN,NaN,E01024509,homepine house,NaN,True,True,"125, SANDGATE ROAD, FOLKESTONE, KENT"
9,52956,K599843,1,K599843-1,False,Leasehold,NaN,NaN,homepine house,125,...,E07000112,0.0,NaN,NaN,E01024509,homepine house,NaN,True,True,"125, SANDGATE ROAD, FOLKESTONE, KENT"
10,49086,K571690,1,K571690-1,False,Freehold,NaN,NaN,NaN,11,...,E07000112,0.0,E01024537,E01024537,NaN,NaN,NaN,True,True,"11, DUNES ROAD, GREATSTONE, NEW ROMNEY, KENT"
11,52348,K692615,1,K692615-1,False,Freehold,NaN,NaN,NaN,84,...,E07000112,0.0,NaN,NaN,NaN,NaN,NaN,True,True,"GND FLR & BST 84, SANDGATE ROAD, FOLKESTONE, KENT"


In [61]:
ocod_data

,Unnamed: 0,title_number,within_title_id,unique_id,within_larger_title,tenure,unit_id,unit_type,building_name,street_number,...,lsoa11cd,msoa11cd,street_name2,lad11cd,business_counts,lsoa_business,lsoa_street,lsoa_building,paon,lsoa_nested
0,0,CB400630,1,CB400630-1,True,Freehold,NaN,NaN,NaN,2,...,E01015611,NaN,millerway,E06000031,0.0,NaN,E01015611,NaN,NaN,E01015611
1,1,CB400630,2,CB400630-2,True,Freehold,NaN,NaN,NaN,4,...,E01015611,NaN,millerway,E06000031,0.0,NaN,E01015611,NaN,NaN,E01015611
2,2,CB400630,3,CB400630-3,True,Freehold,NaN,NaN,NaN,6,...,E01015611,NaN,millerway,E06000031,0.0,NaN,E01015611,NaN,NaN,E01015611
3,3,CB400630,4,CB400630-4,True,Freehold,NaN,NaN,NaN,8,...,E01015611,NaN,millerway,E06000031,0.0,NaN,E01015611,NaN,NaN,E01015611
4,4,CB400630,5,CB400630-5,True,Freehold,NaN,NaN,NaN,10,...,E01015611,NaN,millerway,E06000031,0.0,NaN,E01015611,NaN,NaN,E01015611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173978,87567,AGL465007,1,AGL465007-1,False,Leasehold,807,flat,heritage tower,118,...,E01004215,E02000891,eastferryroad,E09000030,0.0,NaN,NaN,E01004215,heritage tower,NaN
173979,87568,AGL465195,1,AGL465195-1,False,Leasehold,NaN,NaN,ormond house,4t,...,E01032739,E02000001,queenvictoriastreet,E09000001,0.0,E01032739,E01032739,NaN,NaN,NaN
173980,87569,AGL473126,1,AGL473126-1,False,Leasehold,NaN,NaN,NaN,155,...,E01032739,E02000001,bishopsgate,E09000001,0.0,E01032739,E01032739,NaN,NaN,NaN
173981,87570,AGL475468,1,AGL475468-1,False,Leasehold,NaN,NaN,NaN,NaN,...,E01032739,E02000001,NaN,E09000001,0.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.crosstab(test['property_address'].str.contains(r"(penthouse|flat|apartment)", case = False),
           test['address_match'])

# Classify property type

In [ ]:
ocod_data['class'] = np.select(
    [
        ocod_data['property_address'].str.contains(r"^(land|plot)", case = False),
        ocod_data['property_address'].str.contains(r"^((garage)|(parking(\s)?space)|(parking space)|(car park(ing)?(\sspace)))", case = False),
        ocod_data['property_address'].str.contains(r"^((the airspace)|(airspace))", case = False),
        ocod_data['property_address'].str.contains(r"(penthouse|flat|apartment)", case = False),
        ocod_data['property_address'].str.contains(r"(cinema)|(hotel)|(office)|centre|(\bpub)|(business)|(cafe)|(^shop)|( shop)|(restaurant)|(home)|(^store(s)?\b)|(^storage\\b)|(company)|(ltd)|(limited)|(plc)|(retail)|(leisure)|(industrial)|(hall of)", case = False), 
        (ocod_data['business_counts']==0 )& ocod_data.postcode.notnull()
    ], 
    [
        'land',
        'carpark',
        'airspace',
        'domestic',
        'business',
        'domestic'
    ], 
    default='unknown'
)

In [ ]:
#if we can be relatively sure that all the nested properties are domestic then the amount of unclassified 
#properties drops to 2%
#What would allow us to be sure?
ocod_data.groupby('class').size()

In [ ]:
ocod_data.groupby('class').size()

In [ ]:
ocod_data.groupby('class').size()/ocod_data.shape[0]

In [ ]:
#non of the unknowns have a postcode. I guess this is obvious as if there is no matching VOA postcode you are classed as domestic
pd.crosstab(ocod_data[ocod_data['class']=="unknown"].postcode.notnull(), ocod_data[ocod_data['class']=="unknown"].street_name.notnull())

In [ ]:
pd.crosstab(ocod_data['tenure'], ocod_data['region'].str.lower())#.to_latex() #convert to copyable latex table

In [ ]:
pd.crosstab(ocod_data['class'], ocod_data['region'].str.lower())#.to_latex() #convert to copyable latex table

In [ ]:
pd.crosstab(ocod_data['class'], ocod_data['tenure'])#.to_latex() #convert to copyable latex table

In [ ]:
temp_df = ocod_data[['title_number', 'tenure', 'within_larger_title']].drop_duplicates()

#most of titles containing nested addresses are free hold by about 3/2
pd.crosstab(temp_df['tenure'], temp_df['within_larger_title'])


In [ ]:
#The analysis is based on nested addresses being domestic
temp_df = ocod_data[['title_number', 'tenure', 'property_address']][ocod_data['within_larger_title']==True]
temp_df['is_flat'] = temp_df['property_address'].str.contains(r"(flat|apartment|penthouse|unit)", case = False)

#pd.crosstab(temp_df['tenure'], temp_df['within_larger_title'])

temp_df.groupby('tenure').size()

#Of nested addresses freehold is more common by 3/2 50k to 24k
#most of theproperties are not flats however flats dominate the leasehold section
#flats are 1/3 of nested addresses but make up almost 3/4 of the leashold nested addresses
#note this does not include items marked as units
pd.crosstab(temp_df.tenure, temp_df.is_flat)

In [ ]:
ocod_data[ocod_data.price_paid.notnull()==True].to_csv("/tf/empty_homes_data/ocod_price_paid.csv")
pd.crosstab(ocod_data.within_larger_title, ocod_data.price_paid.notnull())

In [ ]:
#This is interesting as it shows that only about 15% of normal titles are missing a postcode whilst half of within larger title addresses are missing a postcode.
#This is probably because they cover multiple postcodes
pd.crosstab(ocod_data.postcode.notnull(), ocod_data.within_larger_title)

In [ ]:
#64% of the propertys have postcodes/lsoa.
#I need to match the remaining to get an accurate distribution of the properties

ocod_data.postcode.notnull().sum()/ocod_data.shape[0]

## Road matching

## Exploring road matching

Before matching the names I should edit the street names to remove all ' apostraphes as these are very unreliable, I should also remove spaces between 'gate' and whatever comes before it same with 'way'
It may also be an idea to remove 's' that appears at the end of a word but we can try this again later

In [ ]:
#target_lad = 'E09000019'
#test= fuzzy_street_match(ocod_data, voa_businesses, target_lad)


In [ ]:
target_lad = 'E09000019'
test= massaged_street_match(ocod_data, voa_businesses, target_lad)

test.columns#[test['street_name2']].groupby('match').size()

In [ ]:
#all_lads = ocod_data.lad11cd.unique()

#all_lads = [x for x in all_lads if str(x) != 'nan']
##see which roads match a road in voa data set for each local authority
#ocod_road_match= [fuzzy_street_match(ocod_data, voa_businesses, target_lad) for target_lad in all_lads]
#ocod_road_match = pd.concat(ocod_road_match)

In [ ]:
all_lads = ocod_data.lad11cd.unique()

all_lads = [x for x in all_lads if str(x) != 'nan']
#see which roads match a road in voa data set for each local authority
ocod_road_match= [massaged_street_match(ocod_data, voa_businesses, target_lad) for target_lad in all_lads]
ocod_road_match = pd.concat(ocod_road_match)

In [ ]:
ocod_road_match.groupby('match').size()

In [ ]:
test = ocod_road_match2[ocod_road_match2['street_name'] != 'xxxstreet name missingxxx']
pd.crosstab(test['match'], test['class'])

In [ ]:
498/(498+3781)

In [ ]:

ocod_road_match2[(ocod_road_match2['match']==False) & (ocod_road_match2['class'] == "business")& (ocod_road_match2['street_name'] != 'xxxstreet name missingxxx')].to_csv("/tf/empty_homes_data/fuzzy_roads.csv")

In [ ]:

ocod_road_match[(ocod_road_match['similarity']<93) & (ocod_road_match['class'] == "business") & (ocod_road_match['street_name'] != 'xxxstreet name missingxxx')].to_csv("/tf/empty_homes_data/fuzzy_roads.csv")

In [ ]:
ocod_road_match[ocod_road_match['street_name'].str.contains(r"'")][['street_name', 'street_name2', 'matched_road_name' ]]

In [ ]:
#values greater than or equal to 94 seem to be pretty good

#ocod_road_match[ocod_road_match['similarity']==96][['property_address', 'street_name2', 'matched_road_name']]

### creating the aggregated ocod dataset for sampling

In [ ]:
ocod_data_lsoa = ocod_data
ocod_data_lsoa['postcode2'] = ocod_data['postcode'].str.lower().str.replace("\s", "")

ocod_data_lsoa = ocod_data.merge(postcode_district_lookup, 'left', left_on = "postcode2", right_on = "postcode2")

ocod_data_lsoa.groupby(['lad11cd', 'lsoa11', 'msoa11', 'class']).size().reset_index().to_csv("/tf/empty_homes_data/ocod_lsoa.csv")
ocod_data_lsoa.groupby(['lad11cd', 'lsoa11', 'msoa11', 'class', 'within_larger_title']).size().reset_index().to_csv("/tf/empty_homes_data/ocod_lsoa_by_nested_type.csv")

In [ ]:

voa_businesses['postcode2'] = voa_businesses['postcode'].str.lower().str.replace("\s", "")

voa_businesses = voa_businesses.merge(postcode_district_lookup, 'left', left_on = "postcode2", right_on = "postcode2")
#postcode_district_lookup


In [ ]:
voa_businesses.columns

In [ ]:
ocod_district = ocod_data[(ocod_data['district'].str.lower() == 'tower hamlets') & (ocod_data['class'] == "unknown") & ocod_data.street_name.notnull() ]

ocod_district = ocod_district[['title_number', 'unit_id', 'building_name','street_number', 'street_name', 'property_address' ]]
voa_district = voa_businesses[voa_businesses['lad11nm'].str.lower() =='tower hamlets']



In [ ]:
ocod_data.groupby('district').size().to_csv('/tf/empty_homes_data/ocod_districts.csv')

In [ ]:
ocod_district[~ocod_district.street_name.str.strip().isin(voa_district.street.str.lower().unique())].to_csv("/tf/empty_homes_data/delete_me.csv")

In [ ]:
pd.crosstab(voa_businesses.postcode.isnull(),voa_businesses.postcode.isnull())

## Largest nested addresses

In [ ]:
#The largest nested address
ocod_data.within_title_id.max()

In [ ]:
ocod_data[ocod_data.within_title_id==ocod_data.within_title_id.max()].reset_index()['property_address'][0]



In [ ]:
ocod_data.to_csv("/tf/empty_homes_data/OCOD_classes.csv")

In [ ]:
test  = ocod_data[ocod_data['class']=="unknown" ]

pd.crosstab(test.postcode.notnull(), test.street_name.notnull())

In [ ]:
ocod_data[ ocod_data.street_name.isnull()].to_csv("/tf/empty_homes_data/OCOD_no_street.csv")

In [ ]:
1/21